In [1]:
%matplotlib inline

import dicom as dc
import numpy as np
import Data_Preprocessing as dp
import matplotlib
import matplotlib.pyplot as plt
import glob
import pandas as pd
import xml
from xml.etree import ElementTree
from shapely.geometry import Polygon
from shapely.geometry import Point

pre_pro = dp.Data_Preprocessing()
path, xml_path = pre_pro.get_random_path()

C:\Users\Seb\Anaconda3\envs\rootclone\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [72]:
path, xml_path = pre_pro.get_random_path()
f = ElementTree.parse(xml_path).getroot()

for child in f.findall('{http://www.nih.gov}ResponseHeader'):
    for grandchild in child.findall('{http://www.nih.gov}SeriesInstanceUid'):
        SeriesUID = grandchild.text
        
dc_file = dc.read_file(path)
z_position = float((dc_file[0x0020, 0x0032].value)[2])
print(z_position)
size = (dc_file.pixel_array).shape
print(size)
annotation_list = []
nodule_list = []
for child in f.findall('{http://www.nih.gov}readingSession'):
    # the annotation mask of this radiologist
    annotations = np.zeros(shape=size)
    nodules = np.zeros(shape=size)
    # loop over nodules
    for grandchild in child.findall('{http://www.nih.gov}unblindedReadNodule'):
        # loop over 2-dim slices of a single nodule
        for ggc in grandchild.findall('{http://www.nih.gov}roi'):
            image_z = float(ggc[0].text)
            #check if current slice has correct z coordinate
            if image_z == z_position:
                print('Matching nodule found')
                vertices = []
                for coord in ggc.findall('{http://www.nih.gov}edgeMap'):
                    vertices.append((int(coord[0].text), int(coord[1].text)))
                    annotations[int(coord[0].text), int(coord[1].text)] = 1
                poly = Polygon(vertices)
                bnd = poly.bounds
                for x in range(int(bnd[0]), int(bnd[2]+1)):
                    for y in range(int(bnd[1]), int(bnd[3] + 1)):     
                        point = Point(x, y)
                        if point.within(poly):
                            nodules[x,y] = 1
    annotation_list.append(annotations)
    nodule_list.append(nodules)

-207.5
(512, 512)


In [2]:
path, xml_path = pre_pro.get_random_path()
pic, anno, nod = pre_pro.get_data(path, xml_path)

KeyError: (0020, 0032)

In [69]:
# renormalize pic
pic = pic - np.amin(pic)
pic = pic/np.amax(pic)
print(np.amin(pic))
print(np.amax(pic))

0.0
1.0


In [73]:
size = pic.shape
three_c = np.zeros(shape=[size[0], size[1], 3])
for k in range(3):
    three_c[...,k] = pic
    
# set red channel to 1 whenever in nodules and set all other channels to 0
for x in range(size[0]):
    for y in range(size[1]):
        if (nod[0])[x,y]==1:
            three_c[x,y,0] = 1
            three_c[x,y,1] = 0
            three_c[x,y,2] = 0
    
three_c[...,0] = three_c[...,0] + nod[0]
plt.figure()
plt.imshow(three_c)
plt.savefig('Data/Test/' + str(20) + '.jpg')
plt.close()

In [9]:
f = ElementTree.parse(xml_path).getroot()
for c in f:
    print (c.tag)
session = f.findall('{http://www.nih.gov}CXRreadingSession')
session

{http://www.nih.gov/idri}ResponseHeader
{http://www.nih.gov/idri}CXRreadingSession
{http://www.nih.gov/idri}CXRreadingSession
{http://www.nih.gov/idri}CXRreadingSession
{http://www.nih.gov/idri}CXRreadingSession


[]